# View files

You can start viewing each file to see what they look correct or additional data is needed.

Let's first load in the data

,image_file_path,raw_annotated_file_path
0,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/ants/diurna...
1,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/fiddlercrab...
2,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/fiddlercrab...
3,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/fiddlercrab...
4,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/fiddlercrab...
...,...,...
152,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,NaN
153,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,NaN
154,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,NaN
155,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,NaN


First, we need to define functions that 